In [30]:
import pandas as pd

In [32]:
train = pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [34]:
X = train.drop('Survived', axis=1)
y = train['Survived']

In [35]:
X.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

<h3>1.</h3>
<pre>Add columns:
    multiple_cabin: '0' if passenger has only 1 cabin, '1' if more than 1
    numeric_ticket: '0' if numeric, '1' otherwise
    name_title: ex. 'Mr', 'Miss' ...
    norm_sibsp: logNormalized 'Fare'
Convert to str: 'Pclass'
Dummy columns: 'Pclass','Sex','Age','SibSp','Parch','norm_fare', 'cabin_adv','cabin_multiple','numeric_ticket','name_title','train_test']
</pre>

<!-- columns to add
training['multiple_cabin'] = all_data.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
training['numeric_ticket'] = training.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
training['name_title'] = training.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())-->

In [36]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.pipeline import TransformerMixin

In [37]:
class DummyEncoder(TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
    def transform(self, X, y=None, **kwargs):
        return pd.get_dummies(X, columns=self.columns)
    def fit(self, X, y=None, **kwargs):
        return self
    
    
class CustomTransformer(BaseEstimator):
    def __init__(self):
        pass
    def fit(self, documents, y=None):
        return self
    def transform(self, X):
        x_dataset = X.copy() # to avoid changes to the original dataset
        x_dataset['multiple_cabin'] = x_dataset.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
        x_dataset['numeric_ticket'] = x_dataset.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
        x_dataset['norm_sibsp'] = np.log(x_dataset.SibSp+1)
        x_dataset['Pclass'] = x_dataset.Pclass.astype(str)
        x_dataset = DummyEncoder().transform(x_dataset[['Pclass','Sex','Age',
                                       'SibSp','Parch','Fare',
                                       'multiple_cabin',
                                       'numeric_ticket',]])
        return x_dataset

<h3>2. Pre-processing steps</h3>
<pre>1) Impute columns: 'Age', 'Fare' (there are no NaN values in 'Fare' column in training.csv, however there in test.csv)
</pre>

In [38]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [39]:
pre_process = ColumnTransformer(remainder='passthrough', 
                                transformers=[('impute_fare', SimpleImputer(strategy='median'), ['Age', 'Fare'])])

<h3>Pipeline</h3>
<pre>Steps:
1) Transform columns
2) pre-processing
3) Train Model
</pre>

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

In [58]:
modelPipeline = Pipeline(steps=[('transform_columns', CustomTransformer()),
                                ('pre_processing', pre_process),
                                ('classifier', LogisticRegression(max_iter=1000))
                               ])

In [101]:
modelPipeline.fit(X, y)

Pipeline(steps=[('transform_columns', CustomTransformer()),
                ('pre_processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_fare',
                                                  SimpleImputer(strategy='median'),
                                                  ['Age', 'Fare'])])),
                ('classifier', LogisticRegression(C=0.1, max_iter=1000))])

In [102]:
modelPipeline.score(X, y)

0.7991021324354658

In [13]:
test = pd.read_csv('test.csv')

In [48]:
modelPipeline.predict(test)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

<h3>Cross-Validation</h3>


In [62]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold

In [20]:
params = {'classifier__C':[0.01, 0.1, 1, 10]}

Split the data into 5 shuffled folds, then each fold is splitted into train and test sets.
 The Pipeline is fitted with train and the score is called with test set.

In [65]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    
    modelPipeline.fit(X_train, y_train)
    
    print(modelPipeline.score(X_test, y_test))

0.8156424581005587
0.7808988764044944
0.8314606741573034
0.7640449438202247
0.7921348314606742


#### hyper-parameter tuning

In [147]:
grid = GridSearchCV(modelPipeline, param_grid=params, cv=kfold)
grid.fit(X_train, y_train)

print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)

Best cross-validation accuracy: 0.80
Best parameters:  {'classifier__C': 0.1}
